In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 
import re
import datetime
import time
import json
from __future__ import print_function
import mysql.connector
from mysql.connector import errorcode

## Create a function to crawl information of binance from a website

In [6]:
def crawl_btc_price():
    res = requests.get('https://www.binance.com/en/trade/BTC_BUSD?_from=markets&theme=dark&type=spot')
    #find the btc price using group 'digit.digit | BTCBUSD'
    btc_price = float(re.findall(r'\d+.\d+ | BTCBUSD', res.text)[0].split(" ")[0])
    now = datetime.datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    btc_info = {'Symbol':'BTC','Price':btc_price,'Updated_At':dt_string}
    res.close()
    return btc_info

## Run the crawling function in 3 hours and place the output into a list

In [10]:
btc_list = []
time_start = time.time()
time_duration = 3 * 60 * 60 #set time duration after three hours
while time.time() < time_start + time_duration:
    btc_item = crawl_btc_price()
    print(crawl_btc_price())
    time.sleep(120)
    btc_list.append(btc_item)

{'Symbol': 'BTC', 'Price': 16745.89, 'Updated_At': '19/12/2022 11:46:16'}
{'Symbol': 'BTC', 'Price': 16748.58, 'Updated_At': '19/12/2022 11:48:17'}
{'Symbol': 'BTC', 'Price': 16769.0, 'Updated_At': '19/12/2022 11:50:19'}
{'Symbol': 'BTC', 'Price': 16751.92, 'Updated_At': '19/12/2022 11:52:20'}
{'Symbol': 'BTC', 'Price': 16751.09, 'Updated_At': '19/12/2022 11:54:22'}
{'Symbol': 'BTC', 'Price': 16744.42, 'Updated_At': '19/12/2022 11:56:23'}
{'Symbol': 'BTC', 'Price': 16753.27, 'Updated_At': '19/12/2022 11:58:25'}
{'Symbol': 'BTC', 'Price': 16757.66, 'Updated_At': '19/12/2022 12:00:26'}
{'Symbol': 'BTC', 'Price': 16759.79, 'Updated_At': '19/12/2022 12:02:27'}
{'Symbol': 'BTC', 'Price': 16760.49, 'Updated_At': '19/12/2022 12:04:28'}
{'Symbol': 'BTC', 'Price': 16748.1, 'Updated_At': '19/12/2022 12:06:29'}
{'Symbol': 'BTC', 'Price': 16749.38, 'Updated_At': '19/12/2022 12:08:29'}
{'Symbol': 'BTC', 'Price': 16753.51, 'Updated_At': '19/12/2022 12:10:30'}
{'Symbol': 'BTC', 'Price': 16760.49, 'Up

## Connecting to MySQL using connector

In [67]:
import os;
from dotenv import load_dotenv;

load_dotenv();

dbHost = os.getenv('DB_HOST');
dbUser = os.getenv('DB_USER');
dbPwd = os.getenv('DB_PASSWORD');
dbName = os.getenv('DB_NAME');

cnx = mysql.connector.connect(user=dbUser, password=dbPwd,
                              host=dbHost,
                              database=dbName)
cursor = cnx.cursor()

## Create a function to make an empty table to receive the crawling output

In [71]:
def create_table():
    try:
        print('Creating table btc_info', end='') # name of the new table is 'btc_info'
        cursor.execute("CREATE TABLE IF NOT EXISTS btc_info (symbol VARCHAR(255), price FLOAT(24), updated_at VARCHAR(255))")
        #create btc_info containing three columns relevant to three pieces of information crawled from the web
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

## Create a function to insert crawled data into btc_info table

In [53]:
def insert_table(data_add):
    create_table()
    for dict_item in data_add:
        placeholders = ','.join(['%s'] * len(dict_item))
        columns = ','.join(dict_item.keys())
        sql_add = "INSERT INTO %s (%s) VALUES (%s)" % ('btc_info', columns, placeholders)
        cursor.execute(sql_add, list(dict_item.values()))
    cnx.commit()

In [54]:
insert_table(btc_list) # run the function to insert the data into the table

Creating table btc_infoOK


## Do some queries to extract interesting information

In [62]:
query = ('SELECT price, updated_at FROM btc_info WHERE price >= 16770')
cursor.execute(query)
myresult = cursor.fetchall()
for i in myresult:
    print(i)

(16777.9, '19/12/2022 12:22:37')
(16770.6, '19/12/2022 12:24:39')
(16775.6, '19/12/2022 12:26:39')
(16774.8, '19/12/2022 12:30:41')
(16774.1, '19/12/2022 12:32:42')
(16771.1, '19/12/2022 12:40:46')
(16774.7, '19/12/2022 12:42:47')
(16770.9, '19/12/2022 12:44:48')
(16772.5, '19/12/2022 12:46:49')
(16771.6, '19/12/2022 12:48:50')
(16773.2, '19/12/2022 12:50:51')
(16770.6, '19/12/2022 12:58:56')
(16771.9, '19/12/2022 13:00:57')
(16775.7, '19/12/2022 13:13:03')


## Task done and stop the connection

In [72]:
cursor.close()
cnx.close()